# Example: Text Generation with IMDB Dataset

In this example, we demonstrate how PFM can be used to modify and improve the output sample from the reference text generation model, on a controlled (positive) sentiment review generation task. 

As done in the DPO paper, to perform a controlled evaluation, we adopt a pre-trained sentiment classifier as the preference annotator. In this example, we chose `lvwerra/distilbert-imdb` from the `huggingface` pipeline. We also adopt a pre-trained GPT-2 on the IMDB dataset (`lvwerra/gpt2-imdb`) from the `huggingface` library as our reference model $\pi_{\mathrm{ref}}$.

For our PFM module to be able to modify variable-length text outputs, we employ a T5-based autoencoder (`thesephist/contra-bottleneck-t5-large-wikipedia`) to work with fixed-sized embeddings, and later decode them back to texts.

In [ ]:
import torch
import numpy as np

from dataset import load_imdb_dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = load_imdb_dataset(
    classifier_checkpoint="lvwerra/distilbert-imdb",
    pretrained_checkpoint="lvwerra/gpt2-imdb",
    autoencoder_checkpoint='thesephist/contra-bottleneck-t5-large-wikipedia',
    split='train[:10000]', 
    device=device
)

### Training PFM on IMDB Preference Dataset

Training a flow matching module can be done within a few lines of codes:

In [ ]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

from flow import OptimalTransportConditionalFlowMatching
from models import UNet

flow_model = UNet().to(device)
flow_matching = OptimalTransportConditionalFlowMatching(flow_model, device=device)

trained_model, _ = flow_matching.fit(
    dataset,
    num_epochs=100,
    batch_size=64,
    learning_rate=1e-3,
    conditional=False,
)

### Visualizing the Transported Output Samples

Once the PFM module is trained, it can directly be attached to the generator to adjust the output samples to be more alinged to the preference. 

For our reference model, we employ a pre-trained GPT-2 model on the IMDB dataset. Since our PFM module is trained using the fixed-sized latent embeddings from the T5-based autoencoder, we need an extra encoding and decoding steps to apply PFM. In particular, we apply a learned flow to the outputs encoded by the autoencoder, and then decode the modified outputs back to texts.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from models import BottleneckT5Autoencoder
from dataset import IMDBEvaluator

tokenizer = AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")
model = AutoModelForCausalLM.from_pretrained("lvwerra/gpt2-imdb").to(device)

evaluator = IMDBEvaluator(device=device)
autoencoder = BottleneckT5Autoencoder(
    model_path='thesephist/contra-bottleneck-t5-large-wikipedia', 
    device=device
)

You may try running the below codes multiple times to visualize the outputs from the base (reference) policy and PFM modified outputs.

In [ ]:
input_text = "The movie was"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

num_iter = 5
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 25,
}

outputs = model.generate(**inputs, **generation_kwargs) #.squeeze()[-TXT_OUT_LEN:]
outputs = tokenizer.decode(outputs.squeeze(), skip_special_tokens=True)
print(f"Generated review from GPT-2 pretrained model:\n{outputs}")
print(f"Score: {evaluator(outputs)}\n")

source = autoencoder.encode(outputs).view(1, 1, 32, 32)
for _ in range(num_iter):
    source = flow_matching.compute_target(source, context=None) 

target = autoencoder.decode(source.view(1, -1))
print(f"Improved review using PFM:\n{target}")
print(f"Score: {evaluator(target)}\n")